# EE-374

# Chapter 6

# Chain Virtues

### 6.1 Transactions - Blocks
- Some attributes that differentiate and give security to different components are summarized in Table.6.1

<img src="images/ch061-txs-blocks.png" width="50%">

### 6.2 Safety, Revisited
- So far we've built a protocol that is safe against *collision attacks, forgery, double spending txs, witholding blocks & adversary forks*. But we can spot another vulnerability in the *read query* functionality of a Ledger - nodes/parties $P_1,P_2$ may not report the exact Ledger version at all times. We'll demand more safety propositions we want to accopmlish 
- We'd like both Ledgers to be exactly the same, so they should report the same element/$\text{tx}$ at position $i$ at time $r_1$: $L_{r_1}^{P_1}[i]=L_{r_1}^{P_2}[i]$, this may fail due to either of the two scenarios below: 
    - *(1) Ledger read query when broadcasting hasn't traversed the whole network yet* - $P_1,P_2$ will report different Ledgers if either one is still catching up to update the latest block while its traversing the network (within $\Delta$ time)
        - **Definition 21** (Ledger Safety). *A protocol is safe if it holds that for any two parties $P_1,P_2$ and any two times $r_1,r_2$, it holds that either $L_{r_1}^{P_1}\preccurlyeq L_{r_2}^{P_2}\vee L_{r_2}^{P_2}\preccurlyeq L_{r_1}^{P_1}$*
            - where notation $A\preccurlyeq B$ for the finite sequences $A,B$ means that $B[:|A|]=A$ ie. one equence is contained within the other one, exactly to every element
        - **Lemma 5** (Safe $\Rightarrow$ No double spend (informal)) *In a safe protocol that ensures transaction validity locally, two honest parties can never confirm two colficting transactions*
    - *(2) Ledger read query when parties are in different forks* - if at some point in time $P_1$ is mining on the longest chain and $P_2$ on a temporary fork, they'll report different Ledgers

### 6.3 Honest Convergence
- Short-lived forks occur all the time and we can rely on the *longest chain rule* for resolution, which is guaranteed to happen because of the probability of many consecutive blocks appended on each fork, each within $\Delta$, (ie. when the network mining power is split) decays quickly
    - Forks can only survive as long as split nodes keep mining blocks in parallel & within $\Delta$, since this is unlikely to go on for long we can expect (Fig.6.3):
    - **Lemma 6** (Honest Convergence (HC)) . *In a setting where only honest parties are mining, exactly $\Delta$ time after every convergence opportunity, all honest parties will hold the same chain*
- **Trade-off**: *blockchain performance & security* - An example of parameterizing the network for perfomance is finding the sweet spot for optimizing *honest convergence opportunities* w/o sacrificing block produciton rate (see Fig.6.4)
    - While every successful PoW query is an HC opportunity, at low *block-production rate* $f=npq$, queries are so rare that HC is rare as well. Whereas at high $f$ we have more forked competing queries & we don't allow the network to have sufficient time to resolve forks
<img src="images/ch063-honest-convergence.png" width="50%">
<img src="images/ch063-density-convergence-opportunity.png" width="50%">

### 6.4 Common Prefix
